Установка функции plotly для построения графиков 

In [ ]:
! pip install --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 12.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


Импорт из библиотек функций

In [ ]:
import csv
import pandas as pd
import plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import numpy as np

Благодаря данной функции- gdown, мы можем загрузить данные с гугл диска

In [ ]:
! pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Тут мы указываем название ссылки, которая находится в нашем гугл.диске.

In [ ]:
!gdown 1srg0lOO6UBgZnrTAA7a1h6SP-mNwz4-7

Downloading...
From: https://drive.google.com/uc?id=1srg0lOO6UBgZnrTAA7a1h6SP-mNwz4-7
To: /content/online_retail_listing.csv
100% 89.1M/89.1M [00:00<00:00, 113MB/s]


Метод импорта пандаса не работает из-за разного количества значений в строке, поэтому был реализован самодельный импорт csv файла.

In [ ]:
lines = []

with open('online_retail_listing.csv', newline='', errors='replace') as f:
    reader = csv.reader(f)
    for line in reader:
        lines += [line]
        
data = {key: [] for key in lines[0][0].split(';')}

for line in lines[1:]:
    result = ''
    for i in line:
      result += i
    line_data = result.split(';')
    if len(line_data) < len(data):
        for i in range(len(data) - len(line_data)):
            line_data.append(None)
    for key, elem in zip(data, line_data):
        data[key] += [elem]

df = pd.DataFrame(data)

# Чистим датасет

В датафрейме содержится большое количество битых строк, поэтому все невалидные данные мы удаляем. Удаляем пустые строки в Customer ID.

In [ ]:
df = df.loc[df['Customer ID'] != '']

Изменение типа данных на корректный. У нас все значение являются числами, кроме даты, и мы приводим к одному типу, чтобы было легче работать с данными.

In [ ]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64')
df['Quantity'] = df['Quantity'].astype('int64')
df['Price'] = df['Price'].astype('int64')
df['Customer ID'] = df['Customer ID'].astype('int64')

Убираем значения по столбцу Quantity, Price которые меньше 0


In [ ]:
df = df.loc[df['Quantity'] > 0]

In [ ]:
df = df.loc[df['Price'] > 0]

In [ ]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,695,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-01-12 07:45:00,675,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-01-12 07:45:00,675,13085,United Kingdom
3,489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-01-12 07:45:00,21,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-01-12 07:45:00,125,13085,United Kingdom
...,...,...,...,...,...,...,...,...
1048570,580501,23284,DOORMAT KEEP CALM AND COME IN,2,2011-04-12 13:00:00,825,14546,United Kingdom
1048571,580501,22507,MEMO BOARD RETROSPOT DESIGN,3,2011-04-12 13:00:00,495,14546,United Kingdom
1048572,580502,22469,HEART OF WICKER SMALL,3,2011-04-12 13:15:00,165,16931,United Kingdom
1048573,580502,23489,VINTAGE BELLS GARLAND,2,2011-04-12 13:15:00,289,16931,United Kingdom


# Создаем Метрику

Доход = количество активных клиентов * количество заказов * средний доход на заказ


Cоздание поля YearMonth для простоты создания отчетов и визуализации.
Рассчитать доход для каждой строки и создать новый фрейм данных со столбцами YearMonth — Revenue

In [ ]:
#создание поля YearMonth для простоты создания отчетов и визуализации
df['InvoiceYearMonth'] = df['InvoiceDate'].map(lambda date: 100*date.year + date.month)

# рассчитать доход для каждой строки и создать новый фрейм данных со столбцами YearMonth — Revenue
df['Revenue'] = df['Price'] * df['Quantity']
df_revenue = df.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
df_revenue

,InvoiceYearMonth,Revenue
0,200901,4031468
1,200902,4687941
2,200903,4513487
3,200904,3178102
4,200905,913939
5,200906,2275741
6,200907,3041041
7,200908,4253400
8,200909,2953140
9,200910,3703993


In [ ]:
# Входы по осям X и Y для графика Plotly. Мы используем Scatter для линейных графиков
plot_data = [
    go.Scatter(
        x=df_revenue['InvoiceYearMonth'],
        y=df_revenue['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Используя функцию pct_change(), чтобы увидеть ежемесячное процентное изменение
Прописываем условия, которые выводят определенную метрику

In [ ]:
#используем функцию pct_change() для просмотра ежемесячного процентного изменения
df_revenue['MonthlyGrowth'] = df_revenue['Revenue'].pct_change()

#показ первых 5 строк
# df_revenue.head()

#визуализация - линейный график
plot_data = [
    go.Scatter(
        x=df_revenue.query("InvoiceYearMonth < 201112")['InvoiceYearMonth'],
        y=df_revenue.query("InvoiceYearMonth < 201112")['MonthlyGrowth'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
df_revenue.head()

,InvoiceYearMonth,Revenue,MonthlyGrowth
0,200901,4031468,NaN
1,200902,4687941,0.162837
2,200903,4513487,-0.037213
3,200904,3178102,-0.295865
4,200905,913939,-0.712426


Ежемесячные активные клиенты



В данной метрике мы можем получить активных клиентов (monthly active) за месяц, подсчитав уникальные идентификаторы клиентов.


In [ ]:
#создание кадра данных ежемесячных активных клиентов путем подсчета уникальных идентификаторов клиентов
df_monthly_active = df.groupby('InvoiceYearMonth')['Customer ID'].nunique().reset_index()

# построение вывода
plot_data = [
    go.Bar(
        x=df_monthly_active['InvoiceYearMonth'],
        y=df_monthly_active['Customer ID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Группировка по году и количеству покупок

In [ ]:
df_monthly_sales = df.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

#plot
plot_data = [
    go.Bar(
        x=df_monthly_sales['InvoiceYearMonth'],
        y=df_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Создаем новый фрейм данных для среднего дохода, взяв его среднее значение

In [ ]:
# создаем новый фрейм данных для среднего дохода, взяв его среднее значение
df_monthly_order_avg = df.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

# построить гистограмму
plot_data = [
    go.Bar(
        x=df_monthly_order_avg['InvoiceYearMonth'],
        y=df_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:

# создайте фрейм данных, содержащий идентификатор клиента и дату первой покупки
df_min_purchase = df.groupby('Customer ID').InvoiceDate.min().reset_index()
df_min_purchase.columns = ['Customer ID','MinPurchaseDate']
df_min_purchase['MinPurchaseYearMonth'] = df_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

# объединить первый столбец даты покупки с нашим основным фреймом данных (tx_uk)
df = pd.merge(df, df_min_purchase, on='Customer ID')

# создайте столбец с именем Тип пользователя и назначьте Существующий
#Если год первой покупки пользователя Месяц до выбранного Месяца года счета-фактуры
df['UserType'] = 'New'
df.loc[df['InvoiceYearMonth']>df['MinPurchaseYearMonth'],'UserType'] = 'Existing'

# рассчитайте доход в месяц для каждого типа пользователей
df_user_type_revenue = df.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()

#фильтруем даты и строим результат
df_user_type_revenue = df_user_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112")
plot_data = [
    go.Scatter(
        x=df_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=df_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=df_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=df_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Коэффициент новых клиентов

In [ ]:
# создайте фрейм данных, который показывает соотношение новых пользователей - нам также нужно отбросить значения NA (коэффициент новых пользователей в первый месяц равен 0)
df_user_ratio = df.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['Customer ID'].nunique() / df.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['Customer ID'].nunique() 
df_user_ratio = df_user_ratio.reset_index()
df_user_ratio = df_user_ratio.dropna()

# Результат отоброжается в графике

plot_data = [
    go.Bar(
        x=df_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")['InvoiceYearMonth'],
        y=df_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")['Customer ID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

Ежемесячный показатель удержания
Ежемесячный показатель удержания = количество удержанных клиентов с предыдущего. Месяц/активных клиентов всего

Мы будем использовать функцию crosstab() из pandas, которая делает вычисление Retention Rate очень простым.

In [ ]:
#определить, какие пользователи активны, посмотрев на их ежемесячный доход
df_user_purchase = df.groupby(['Customer ID','InvoiceYearMonth'])['Revenue'].sum().reset_index()

#создать матрицу удержания с перекрестной таблицей
df_retention = pd.crosstab(df_user_purchase['Customer ID'], df_user_purchase['InvoiceYearMonth']).reset_index()

#создать массив словарей, в котором хранится количество удержанных и общих пользователей за каждый месяц
months = df_retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = df_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = df_retention[(df_retention[selected_month] > 0) & (df_retention[prev_month] > 0)][selected_month].sum()
    retention_array.append(retention_data)
    
# преобразовать массив в фрейм данных и рассчитать Retention Rate
df_retention = pd.DataFrame(retention_array)
df_retention['RetentionRate'] = df_retention['RetainedUserCount'] / df_retention['TotalUserCount']

# построить график коэффициента удержания
plot_data = [
    go.Scatter(
        x = df_retention.query("InvoiceYearMonth < 201112")['InvoiceYearMonth'],
        y = df_retention.query("InvoiceYearMonth < 201112")['RetentionRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

**Коэффициент удержания на основе когорты**

Это представление поможет нам увидеть, как недавние и старые когорты отличаются по коэффициенту удержания, и повлияли ли недавние изменения в клиентском опыте на удержание новых клиентов или нет.

In [ ]:
# снова создайте нашу таблицу удержания с помощью crosstab() и добавьте представление по месяцу года первой покупки
df_retention = pd.crosstab(df_user_purchase['Customer ID'], df_user_purchase['InvoiceYearMonth']).reset_index()
df_retention = pd.merge(df_retention,df_min_purchase[['Customer ID','MinPurchaseYearMonth']],on='Customer ID')
new_column_names = [ 'm_' + str(column) for column in df_retention.columns[:-1]]
new_column_names.append('MinPurchaseYearMonth')
df_retention.columns = new_column_names

#создаем массив удержанных пользователей для каждой когорты ежемесячно
retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i + 1:]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan
        
    total_user_count = df_retention[df_retention.MinPurchaseYearMonth ==  selected_month].MinPurchaseYearMonth.count()
    retention_data['TotalUserCount'] = total_user_count
    retention_data[selected_month] = 1 
    
    query = "MinPurchaseYearMonth == {}".format(selected_month)
    

    for next_month in next_months:
        new_query = query + " and {} > 0".format(str('m_' + str(next_month)))
        retention_data[next_month] = np.round(df_retention.query(new_query)['m_' + str(next_month)].sum() / total_user_count, 2)
    retention_array.append(retention_data)
    
df_retention = pd.DataFrame(retention_array)
df_retention.index = months

#показ новой таблицы удержания на основе когорт
df_retention

,TotalUserCount,200902,200903,200904,200905,200906,200907,200908,200909,200910,...,201103,201104,201105,201106,201107,201108,201109,201110,201111,201112
InvoiceYearMonth,,,,,,,,,,,,,,,,,,,,,
200902,91,1.0,0.04,0.04,0.00,0.00,0.04,0.07,0.03,0.05,...,0.29,0.27,0.30,0.33,0.25,0.25,0.36,0.35,0.44,0.16
200903,95,NaN,1.00,0.02,0.00,0.02,0.00,0.05,0.04,0.01,...,0.35,0.34,0.35,0.32,0.31,0.25,0.32,0.35,0.41,0.16
200904,67,NaN,NaN,1.00,0.01,0.01,0.03,0.04,0.03,0.01,...,0.27,0.24,0.30,0.24,0.28,0.36,0.33,0.27,0.36,0.12
200905,25,NaN,NaN,NaN,1.00,0.04,0.04,0.00,0.04,0.00,...,0.24,0.12,0.20,0.20,0.24,0.16,0.32,0.40,0.20,0.12
200906,58,NaN,NaN,NaN,NaN,1.00,0.03,0.00,0.02,0.07,...,0.16,0.22,0.22,0.21,0.17,0.24,0.31,0.34,0.26,0.07
200907,63,NaN,NaN,NaN,NaN,NaN,1.00,0.02,0.03,0.03,...,0.27,0.37,0.35,0.25,0.30,0.22,0.19,0.27,0.38,0.14
200908,69,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.03,0.03,...,0.23,0.19,0.22,0.20,0.19,0.20,0.28,0.26,0.42,0.07
200909,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.00,...,0.31,0.33,0.26,0.22,0.24,0.19,0.22,0.24,0.30,0.15
200910,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,...,0.25,0.18,0.23,0.31,0.30,0.26,0.34,0.33,0.44,0.10
